In [5]:
# we use this function to generate a LUT for our designed Approxiate Multiplier.
import numpy as np

In [6]:
def Approx_Multiply(multOp1, multOp2):
    '''
    The first operand (multOp1) is first converted to (2,5,1) float, Mantissa is multiplied by multOp2.
    The result is then converted back to fixed point.
    '''
    multOp1Abs = np.abs(multOp1)
    if multOp1 < 0:
      multOp1Abs = multOp1Abs - 1
    # Encoding to floating point
    exponent = np.floor(np.log2(multOp1Abs)) - 4
    if exponent < 0:
        exponent = 0
    #if multOp1 >= 0:
    mantissa = np.floor(multOp1Abs / pow(2,exponent))  # 5-bit mantissa
    #else:
    #    mantissa = np.floor(multOp1Abs / pow(2,exponent))  # 5-bit mantissa
    print(exponent, mantissa)
    # Multiplying the Mantissa
    resMantissa = round (mantissa * np.abs(multOp2) / 128)
    # Decoder: back to fixed point
    result = resMantissa * pow(2, exponent)
    # Fixing sign of the result
    if multOp1 * multOp2 < 0:
        result = -result

    return result

In [27]:
def quantize(inOp, rangeMax, bitWidth):
    absBitWidth = bitWidth - 1 # bitwidth without sign
    absRangeMax = np.abs(rangeMax)
    if np.abs(inOp) > absRangeMax + 1:
        print('ERROR: inOp cannot be larger than rangeMax!')
        return -1
    if rangeMax == 0:
        print('ERROR: rangeMax cannot be zero!')
        return -1
    elif rangeMax < pow(2,absBitWidth):
        shiftSize = absBitWidth - np.floor(np.log2(absRangeMax) + 1)
        quantizedResult = round(inOp * pow(2,shiftSize))  # Scaling and rounding
    else:
        shiftSize = np.floor(np.log2(absRangeMax) + 1) - absBitWidth
        quantizedResult = round(inOp / pow(2,shiftSize))

    return quantizedResult

    print('--', shiftSize, quantizedResult, np.floor(np.log2(np.abs(rangeMax))))


multOp1 = 126
multOp2 = -128
rangeMax = 127
multOp1Q = quantize(multOp1, rangeMax, 8)
multOp2Q = quantize(multOp2, rangeMax, 8)
print(f'--- rangeMax = {rangeMax}   Operand = {multOp1}  Quantized Operand  = {multOp1Q}')
print(f'--- rangeMax = {rangeMax}   Operand = {multOp2}  Quantized Operand = {multOp2Q}')

print(f'mult result = {Approx_Multiply(multOp1Q, multOp2Q)}')

--- rangeMax = 127   Operand = 126  Quantized Operand  = 126
--- rangeMax = 127   Operand = -128  Quantized Operand = -128
2.0 31.0
mult result = -124.0


In [8]:
#Generate LUT and save them as .h and .csv
def Generate_LUT(bitwidth_Op1, bitwidth_Op2):

  NPV1 = pow(2, bitwidth_Op1)        #Number of possible values for Op1
  NPV2 = pow(2, bitwidth_Op2)        #Number of possible values for Op2
  LUT = np.zeros((NPV1, NPV2))

  for i in range(NPV1):
    for j in range(NPV2):
      LUT[i, j] = Approx_Multiply(i- pow(2, bitwidth_Op1- 1), j- pow(2, bitwidth_Op2- 1))
      #LUT[i, j] = Approx_Multiply(quantize(i- pow(2, bitwidth_Op1- 1), pow(2, bitwidth_Op1- 1)-1, bitwidth_Op1), quantize(j- pow(2, bitwidth_Op2- 1), pow(2, bitwidth_Op2- 1)-1, bitwidth_Op2))      #if you want to quantize Ops before Multiplying

  Save_LUT_CSV(LUT)
  Save_LUT_C()
  return LUT



In [9]:
#Make a .h file containg Approximate Multiplier LUT
def Save_LUT_C():
  nbits=8
  with open('LUT_C'+'.h', 'w') as myfile:
    bits = int(pow(2,nbits))
    lut_size_str = str(bits)

    myfile.write('#include <stdint.h>\n\n')
    myfile.write('const int' + str(2*nbits) + '_t lut [' + lut_size_str + '][' + lut_size_str +'] = {')

    for i in range (0,bits//2):
      myfile.write('{')
      for j in range (0,bits//2):
        x = int(Approx_Multiply(i,j) * 128)
        myfile.write('%s' % x)
        myfile.write(', ')
      for j in range (bits//2,bits):
        x = int(Approx_Multiply(i,(bits-j)*-1) * 128)
        myfile.write('%s' % x)
        if j!=bits-1:
          myfile.write(', ')
      myfile.write('},')
      myfile.write('\n')

    for i in range (bits//2,bits):
        myfile.write('{')
        for j in range (0,bits//2):
            x = int(Approx_Multiply((bits-i)*-1,j) * 128)
            myfile.write('%s' % x)
            myfile.write(', ')
        for j in range (bits//2,bits):
            x = int(Approx_Multiply((bits-i)*-1,(bits-j)*-1) * 128)
            myfile.write('%s' % x)
            if j!=bits-1:
                myfile.write(', ')
        if(i!=bits-1):
            myfile.write('},')
            myfile.write('\n')
    myfile.write('}};\n')
  return

In [10]:
#Make a .csv file containg Approximate Multiplier LUT
import numpy as np
def Save_LUT_CSV(LUT):
  np.savetxt('LUT.csv', LUT, delimiter=',')
  return


In [11]:
F= Generate_LUT(8, 8)

Streaming output truncated to the last 5000 lines.
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 20.0
0.0 

<ipython-input-6-0b5759a3e3a3>:10: RuntimeWarning: divide by zero encountered in log2
  exponent = np.floor(np.log2(multOp1Abs)) - 4


Streaming output truncated to the last 5000 lines.
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 19.0
0.0 

In [12]:
#Loading_LUT_CVS
LUT_loaded = np.loadtxt('LUT.csv', delimiter=',')
print(np.array_equal(F, LUT_loaded))

True


In [13]:
print(LUT_loaded[0,:])

[ 124.  124.  124.  120.  120.  120.  120.  116.  116.  116.  116.  112.
  112.  112.  112.  108.  108.  108.  108.  104.  104.  104.  104.  100.
  100.  100.  100.   96.   96.   96.   96.   92.   92.   92.   92.   92.
   88.   88.   88.   88.   84.   84.   84.   84.   80.   80.   80.   80.
   76.   76.   76.   76.   72.   72.   72.   72.   68.   68.   68.   68.
   64.   64.   64.   64.   64.   60.   60.   60.   60.   56.   56.   56.
   56.   52.   52.   52.   52.   48.   48.   48.   48.   44.   44.   44.
   44.   40.   40.   40.   40.   36.   36.   36.   36.   32.   32.   32.
   32.   32.   28.   28.   28.   28.   24.   24.   24.   24.   20.   20.
   20.   20.   16.   16.   16.   16.   12.   12.   12.   12.    8.    8.
    8.    8.    4.    4.    4.    4.    0.    0.    0.   -0.   -0.   -4.
   -4.   -4.   -4.   -8.   -8.   -8.   -8.  -12.  -12.  -12.  -12.  -16.
  -16.  -16.  -16.  -20.  -20.  -20.  -20.  -24.  -24.  -24.  -24.  -28.
  -28.  -28.  -28.  -32.  -32.  -32.  -32.  -32.  -

In [14]:
with open('LUT_C.h') as my_file:
    for line in my_file:
        print(line)

#include <stdint.h>



const int16_t lut [256][256] = {{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0},

{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
print(F.shape)

(256, 256)
